In [3]:
import pandas as pd
import numpy as np
import csv


d_train=pd.read_csv("/Users/lucasmuzynoski/Projects/Numerai2/numerai_datasets/numerai_training_data.csv")
"""
all_eras=set(np.unique(d_train["era"]))
validate_eras={"era100","era101","era102","era103","era104","era105","era106","era107","era108",
               "era109","era110","era111","era112","era113","era114","era115","era116","era117","era118","era119","era120"}
train_eras=all_eras-validate_eras
d_val=d_train[d_train["era"].isin(validate_eras)]

d_train=d_train[d_train["era"].isin(train_eras)]
"""
d_val=pd.read_csv("/Users/lucasmuzynoski/Projects/Numerai2/numerai_datasets/numerai_tournament_data.csv")
d_val=d_val[d_val["data_type"]=="validation"]
feature_names = [
        f for f in d_train.columns if f.startswith("feature")
    ]


from sklearn.neural_network import MLPRegressor
from random import sample, choices
from scipy.stats import spearmanr
from sklearn.linear_model import ElasticNetCV, SGDRegressor
from scipy.optimize import minimize_scalar
from sklearn.decomposition import PCA

In [4]:
def base_learner(X,y):
    nn_mod=MLPRegressor(hidden_layer_sizes=(24),alpha=.001,n_iter_no_change=1,learning_rate_init=.001,tol=.00001,
                        activation='tanh',shuffle=False)
    nn_mod.fit(X,y)
    return(nn_mod)

def neg_grad_calc_(x,y):
    epsilon=.125
    grad=np.where(abs(x-y)<epsilon,0,np.where(x>y+epsilon,-1,1))
    return(grad)

def eval_metric_calc_(y_pred,y):
    
    return(spearmanr(y_pred,y))

def loss_func(y,y_pred):
    epsilon=.125
    loss=np.where(abs(y-y_pred)<epsilon,0,abs(y-y_pred)-epsilon)
    total_loss=np.sum(loss)
    return(total_loss)


def nn_boost_fit(X,y,X_val,y_val,eta=.2,n_estimators=100,verbose=True,col_sample_num=10,subsample=.75):
    base_learner_list=[]
    input_list=[]
    rho_list=[]
    mu=np.mean(y)

    #initialize with a linear model
    f_0=SGDRegressor(loss="epsilon_insensitive",epsilon=.125,penalty='elasticnet',early_stopping=True)
    #f_0=ElasticNetCV(n_jobs=-1)
    f_0.fit(X,y)
    f_curr=f_0.predict(X)
    f_val=f_0.predict(X_val)
    
    
    #f_0=np.repeat(mu,X.shape[0])
    #f_curr=f_0
    #f_val=np.repeat(mu,X_val.shape[0])
    
    #col_sample_num=round(col_sample*X.shape[1])
    #col_sample_num=1
    row_sample_num=round(subsample*X.shape[0])
    #rho_threshold=0

    for i in range(n_estimators):
        #if (i % 5)==0: 
        #    neg_grad_total=pd.Series(neg_grad_calc_(f_curr,y))
        #   w=np.abs(X.corrwith(neg_grad_total,method="spearman"))
        #rho=-1
        
        col_samp=sample(list(X.columns),k=col_sample_num)
        input_list.append(col_samp)
        row_samp=choices(range(X.shape[0]),k=row_sample_num)
        #X_samp=X.iloc[row_samp,col_samp]
        X_samp=X.loc[row_samp,col_samp]
        y_samp=y.loc[row_samp]
        neg_grad_samp=neg_grad_calc_(f_curr[row_samp],y_samp)
        base_learner_list.append(base_learner(X_samp,neg_grad_samp))
        h_samp=base_learner_list[i].predict(X_samp)
        h=base_learner_list[i].predict(X[col_samp])
        h_val=base_learner_list[i].predict(X_val[col_samp])
            
    
        #rho=np.cov(h,y-f_curr)[0,1]/np.var(h)  #only works for l2 loss currently

        f_curr_samp=f_curr[row_samp]

        def loss(rho):
            y_pred=f_curr+eta*rho*h
            loss=loss_func(y,y_pred)
            return loss

        rho_optim=minimize_scalar(fun=loss,method="brent")
        rho=rho_optim.x

        rho_list.append(rho)
        f_curr=f_curr+eta*rho*h
        f_val=f_val+eta*rho*h_val

        if verbose:
            print(eval_metric_calc_(f_val,y_val))

    return(base_learner_list,input_list,rho_list,mu)


In [5]:
nn_boost_mod=nn_boost_fit(X=d_train[feature_names],y=d_train["target"],X_val=d_val[feature_names],y_val=d_val["target"],n_estimators=75)

SpearmanrResult(correlation=0.011782040615625926, pvalue=1.223039294661651e-05)
SpearmanrResult(correlation=0.013453313801565139, pvalue=5.918382899601806e-07)
SpearmanrResult(correlation=0.013432717536359882, pvalue=6.157344816629027e-07)
SpearmanrResult(correlation=0.012245694633322252, pvalue=5.478969373307616e-06)
SpearmanrResult(correlation=0.01261283297064089, pvalue=2.84297537533144e-06)
SpearmanrResult(correlation=0.012216044273022465, pvalue=5.772591057509733e-06)
SpearmanrResult(correlation=0.012545470111887747, pvalue=3.210949995572228e-06)
SpearmanrResult(correlation=0.012313087570749072, pvalue=4.863837888299346e-06)
SpearmanrResult(correlation=0.015038025621846095, pvalue=2.3753792949648754e-08)
SpearmanrResult(correlation=0.014770809870081726, pvalue=4.182528434788236e-08)
SpearmanrResult(correlation=0.015595879972137988, pvalue=7.069262126612024e-09)
SpearmanrResult(correlation=0.01585077939569296, pvalue=4.007036261089204e-09)
SpearmanrResult(correlation=0.016678422925